In [266]:
// Data Types and Functions
#nullable enable

using System.Text.RegularExpressions;

public record struct Position {
    public int X;
    public int Y;

    public static Position operator+(Position position, Position other) {
        return new Position {
            X = position.X + other.X,
            Y = position.Y + other.Y,
        };
    }
    
    public static Position operator-(Position position, Position other) {
        return new Position {
            X = position.X - other.X,
            Y = position.Y - other.Y,
        };
    }
}

public enum Direction {
    Up,
    Down,
    Left,
    Right
}

public static Direction ParseDirection(string input) {
    return input switch {
        "U" => Direction.Up,
        "D" => Direction.Down,
        "L" => Direction.Left,
        "R" => Direction.Right,
        _ => throw new System.InvalidOperationException("Unknown direction"),
    };
}

public static Position Delta(Direction direction) {
    return direction switch {
        Direction.Up => new Position { X = 0, Y = -1 },
        Direction.Down => new Position { X = 0, Y = 1 },
        Direction.Left => new Position { X = -1, Y = 0 },
        Direction.Right => new Position { X = 1, Y = 0 },
    };
}

public static int Value(Direction direction) {
    return direction switch {
        Direction.Up => 3,
        Direction.Left => 2,
        Direction.Down => 1,
        Direction.Right => 0,
    };
}

public enum TurnDirection {
    Left,
    Right,
}

public static Direction ApplyTurn(Direction original, TurnDirection turn) {
    var directions = new List<Direction> {
        Direction.Up,
        Direction.Right,
        Direction.Down,
        Direction.Left,
    };
    var index = turn switch {
        TurnDirection.Left => (directions.IndexOf(original) + directions.Count - 1) % directions.Count,
        TurnDirection.Right => (directions.IndexOf(original) + 1) % directions.Count,
    };
    return directions[index];
}

public record struct Instruction {
    public int? WalkAmount;
    public TurnDirection? TurnDirection;
}

public enum Space {
    Empty,
    Wall,
}

static Dictionary<Position, Space> ParseMap(string input) {
    return input.Split("\n")
        .SelectMany(
            (line, y) => line.Select((c, x) => (
                Position: new Position {
                    X = x,
                    Y = y,
                },
                Space: c switch {
                    '.' => Space.Empty,
                    '#' => Space.Wall,
                    ' ' => (Space?) null,
                    _ => throw new InvalidOperationException($"Invalid map character {c}"),
                }
            ))
        )
        .Where(tuple => tuple.Space != null)
        .ToDictionary(tuple => tuple.Position, tuple => (Space) tuple.Space);
}

static readonly var NumberRegex = new Regex(@"(\d+)");
static readonly var TurnDirectionRegex = new Regex(@"([LR])");
public static Instruction ParseInstruction(string input) {
    if (NumberRegex.IsMatch(input)) {
        return new Instruction {
            WalkAmount = int.Parse(input),
        };
    }
    if (TurnDirectionRegex.IsMatch(input)) {
        return new Instruction {
            TurnDirection = input switch {
                "L" => TurnDirection.Left,
                "R" => TurnDirection.Right,
                _ => throw new InvalidOperationException($"Invalid turn direction {input}"),
            }
        };
    }
    throw new InvalidOperationException($"Invalid input {input}");
}

static readonly var NumberOrTurnDirectionRegex = new Regex(@"(\d+|[LR])");
public static List<Instruction> ParseInstructions(string input) {
    return NumberOrTurnDirectionRegex.Matches(input)
        .Select(match => ParseInstruction(match.Value))
        .ToList();
}

(Dictionary<Position, Space> Map, List<Instruction> Instructions) ParseInput(string input) {
    var parts = input.Split("\n\n");
    return (
        ParseMap(parts[0]),
        ParseInstructions(parts[1])
    );
}

public record struct State {
    public Position Position;
    public Direction Direction;
}
public static State ApplyWalkInstruction(Dictionary<Position, Space> map, State state, int walkAmount) {
    var currentPosition = state.Position;
    for (int i = 0; i < walkAmount; i++) {
        var nextPosition = currentPosition + Delta(state.Direction);
        if (!map.ContainsKey(nextPosition)) {
            nextPosition = state.Direction switch {
                Direction.Down => map.Keys
                                    .Where(p => p.X == currentPosition.X)
                                    .MinBy(p => p.Y),
                Direction.Up => map.Keys
                                    .Where(p => p.X == currentPosition.X)
                                    .MaxBy(p => p.Y),
                Direction.Right => map.Keys
                                    .Where(p => p.Y == currentPosition.Y)
                                    .MinBy(p => p.X),
                Direction.Left => map.Keys
                                    .Where(p => p.Y == currentPosition.Y)
                                    .MaxBy(p => p.X),
            };
        }
        if (map[nextPosition] == Space.Wall) {
            break;
        } else {
            currentPosition = nextPosition;
        }
    }
    return state with {
        Position = currentPosition,
    };
}
public static State ApplyTurnInstruction(State state, TurnDirection turn) {
    return state with {
        Direction = ApplyTurn(state.Direction, turn),
    };
}


public static State ApplyInstruction(Dictionary<Position, Space> map, State state, Instruction instruction) {
    if (instruction.WalkAmount != null) {
        return ApplyWalkInstruction(map, state, instruction.WalkAmount.Value);
    }
    if (instruction.TurnDirection != null) {
        return ApplyTurnInstruction(state, instruction.TurnDirection.Value);
    }
    throw new InvalidOperationException("Invalid instruction");
}


In [267]:
// Read input
var input = System.IO.File.ReadAllText("input.txt");
var sideLength = 50;
input

                                                  ...#..#.#........#.#..#......#..#.#....##................##.#...#....#.............#........#.......
                                                  ...#..........#.......................#...#.##.................#.............#.......#...#..........
                                                  ........##....#..#....................#.......#.........#............#..#....#......................
                                                  ..#..###..........................#..#...#.....##.........#..............##...#........#............
                                                  #..#........#............#..#........#..##.....#.....#.#............##.......#.........###...#..#...
                                                  ...................#...............#....#...................#.......#...........................#...
                                                  ..#......................#....#......#....#.

In [268]:
// Part 1
var (map, instructions) = ParseInput(input);
var startingPosition = map.Keys
    .Where(p => p.Y == 0)
    .MinBy(p => p.X);
var startingState = new State {
    Position = startingPosition,
    Direction = Direction.Right,
};

var finalState = instructions.Aggregate(startingState, (state, instruction) => ApplyInstruction(map, state, instruction));
(finalState.Position.Y + 1) * 1000 + (finalState.Position.X + 1) * 4 + Value(finalState.Direction)

186128

In [280]:
// Part 2 Data Types and Functions
public enum CubeRotation {
    Up,
    Down,
    Right,
    Left
}

public static CubeRotation Invert(CubeRotation rotation) {
    return rotation switch {
        CubeRotation.Up => CubeRotation.Down,
        CubeRotation.Down => CubeRotation.Up,
        CubeRotation.Left => CubeRotation.Right,
        CubeRotation.Right => CubeRotation.Left,
    };
}

public enum FaceRotation {
    Clockwise,
    CounterClockwise,
}

public static Dictionary<Position, FacePixel> RotateFace(Dictionary<Position, FacePixel> face, FaceRotation direction) {
    // Assumption: square faces, 0->N
    var maxValue = face.Keys.Max(p => p.X);
    
    return direction switch {
        FaceRotation.Clockwise => face.ToDictionary(
            kvp => new Position {
                X = maxValue - kvp.Key.Y,
                Y = kvp.Key.X,
            },
            kvp => kvp.Value
        ),
        FaceRotation.CounterClockwise => face.ToDictionary(
            kvp => new Position {
                X = kvp.Key.Y,
                Y = maxValue - kvp.Key.X,
            },
            kvp => kvp.Value
        ),
    };
}

public static string DisplayFace(Dictionary<Position, FacePixel> face) {
    var sideLength = face.Keys.Max(p => p.X) + 1;
    var buffer = new StringBuilder();
    for (var y = 0; y < sideLength; y++) {
        for (var x = 0; x < sideLength; x++) {
            var c = face[new Position { X = x, Y = y }].Content switch {
                Space.Empty => ".",
                Space.Wall => "#",
            };
            buffer.Append(c);
        }
        buffer.Append("\n");
    }
    return buffer.ToString();
}

public record struct Cube {
    public Dictionary<Position, FacePixel> Front;
    public Dictionary<Position, FacePixel> Back;
    public Dictionary<Position, FacePixel> Left;
    public Dictionary<Position, FacePixel> Right;
    public Dictionary<Position, FacePixel> Top;
    public Dictionary<Position, FacePixel> Bottom;

    public Cube Rotate(CubeRotation direction) {
        return direction switch {
            CubeRotation.Up => new Cube {
                Top = Front,
                Bottom = Back,
                Left = RotateFace(Left, FaceRotation.CounterClockwise),
                Right = RotateFace(Right, FaceRotation.Clockwise),
                Back = Top,
                Front = Bottom,
            },
            CubeRotation.Down => new Cube {
                Top = Back,
                Bottom = Front,
                Left = RotateFace(Left, FaceRotation.Clockwise),
                Right = RotateFace(Right, FaceRotation.CounterClockwise),
                Back = Bottom,
                Front = Top,
            },
            CubeRotation.Left => new Cube {
                Left = Front,
                Right = RotateFace(RotateFace(Back, FaceRotation.Clockwise), FaceRotation.Clockwise),
                Top = RotateFace(Top, FaceRotation.Clockwise),
                Bottom = RotateFace(Bottom, FaceRotation.CounterClockwise),
                Front = Right,
                Back = RotateFace(RotateFace(Left, FaceRotation.Clockwise), FaceRotation.Clockwise),
            },
            CubeRotation.Right => new Cube {
                Left = RotateFace(RotateFace(Back, FaceRotation.Clockwise), FaceRotation.Clockwise),
                Right = Front,
                Top = RotateFace(Top, FaceRotation.CounterClockwise),
                Bottom = RotateFace(Bottom, FaceRotation.Clockwise),
                Front = Left,
                Back = RotateFace(RotateFace(Right, FaceRotation.Clockwise), FaceRotation.Clockwise),
            },
        };
    }
}

public record struct PartialCube {
    public Dictionary<Position, FacePixel>? Front;
    public Dictionary<Position, FacePixel>? Back;
    public Dictionary<Position, FacePixel>? Left;
    public Dictionary<Position, FacePixel>? Right;
    public Dictionary<Position, FacePixel>? Top;
    public Dictionary<Position, FacePixel>? Bottom;

    public PartialCube Rotate(CubeRotation direction) {
        return direction switch {
            CubeRotation.Up => new PartialCube {
                Top = Front,
                Bottom = Back,
                Left = Left == null ? null : RotateFace(Left, FaceRotation.CounterClockwise),
                Right = Right == null ? null : RotateFace(Right, FaceRotation.Clockwise),
                Back = Top,
                Front = Bottom,
            },
            CubeRotation.Down => new PartialCube {
                Top = Back,
                Bottom = Front,
                Left = Left == null ? null : RotateFace(Left, FaceRotation.Clockwise),
                Right = Right == null ? null : RotateFace(Right, FaceRotation.CounterClockwise),
                Back = Bottom,
                Front = Top,
            },
            CubeRotation.Left => new PartialCube {
                Left = Front,
                Right = Back == null ? null : RotateFace(RotateFace(Back, FaceRotation.Clockwise), FaceRotation.Clockwise),
                Top = Top == null ? null : RotateFace(Top, FaceRotation.Clockwise),
                Bottom = Bottom == null ? null : RotateFace(Bottom, FaceRotation.CounterClockwise),
                Front = Right,
                Back = Left == null ? null : RotateFace(RotateFace(Left, FaceRotation.Clockwise), FaceRotation.Clockwise),
            },
            CubeRotation.Right => new PartialCube {
                Left = Back == null ? null : RotateFace(RotateFace(Back, FaceRotation.Clockwise), FaceRotation.Clockwise),
                Right = Front,
                Top = Top == null ? null : RotateFace(Top, FaceRotation.CounterClockwise),
                Bottom = Bottom == null ? null : RotateFace(Bottom, FaceRotation.Clockwise),
                Front = Left,
                Back = Right == null ? null : RotateFace(RotateFace(Right, FaceRotation.Clockwise), FaceRotation.Clockwise),
            },
        };
    }

    public bool IsWhole() => Front != null
        && Back != null
        && Left != null
        && Right != null
        && Top != null
        && Bottom != null;
    
    public Cube GetWholeCube() => new Cube {
        Front = Front,
        Back = Back,
        Left = Left,
        Right = Right,
        Top = Top,
        Bottom = Bottom,
    };
}

static PartialCube ExplorePartialCube(Dictionary<Position, Dictionary<Position, FacePixel>> regions, Position currentRegion, PartialCube cubeSoFar) {
    cubeSoFar.Front = regions[currentRegion];
    var topRegion = currentRegion + new Position { X = 0, Y = -1};
    if (cubeSoFar.Top == null && regions.ContainsKey(topRegion)) {
        cubeSoFar = ExplorePartialCube(regions, topRegion, cubeSoFar.Rotate(CubeRotation.Down))
            .Rotate(CubeRotation.Up);
    }
    var leftRegion = currentRegion + new Position { X = -1, Y = 0};
    if (cubeSoFar.Left == null && regions.ContainsKey(leftRegion)) {
        cubeSoFar = ExplorePartialCube(regions, leftRegion, cubeSoFar.Rotate(CubeRotation.Right))
            .Rotate(CubeRotation.Left);
    }
    var rightRegion = currentRegion + new Position { X = 1, Y = 0};
    if (cubeSoFar.Right == null && regions.ContainsKey(rightRegion)) {
        cubeSoFar = ExplorePartialCube(regions, rightRegion, cubeSoFar.Rotate(CubeRotation.Left))
            .Rotate(CubeRotation.Right);
    }
    var bottomRegion = currentRegion + new Position { X = 0, Y = 1};
    if (cubeSoFar.Bottom == null && regions.ContainsKey(bottomRegion)) {
        cubeSoFar = ExplorePartialCube(regions, bottomRegion, cubeSoFar.Rotate(CubeRotation.Up))
            .Rotate(CubeRotation.Down);
    }
    return cubeSoFar;
}

public record struct FacePixel {
    public Space Content;
    public Position MapPosition;
}

static Cube CubeFromMap(Dictionary<Position, Space> map, int sideLength) {
    var regions = map.GroupBy(kvp => new Position { X = kvp.Key.X / sideLength, Y = kvp.Key.Y / sideLength })
        .ToDictionary(
            group => group.Key,
            group => group.ToDictionary(
                item => new Position { X = item.Key.X % sideLength, Y = item.Key.Y % sideLength },
                item => new FacePixel {
                    Content = item.Value,
                    MapPosition = item.Key,
                }
            )
        );
    var frontRegion = regions.Keys
        .Where(regionLocation => regionLocation.Y == 0)
        .MinBy(regionLocation => regionLocation.X);
    
    var partialCube = new PartialCube{};
    var finalCube = ExplorePartialCube(regions, frontRegion, partialCube);
    if (!finalCube.IsWhole()) {
        throw new InvalidOperationException("Failed to explore cube map");
    }
    return finalCube.GetWholeCube();
}

public record struct StatePart2 {
    public Cube Cube;
    public Position Position;
    public Direction Direction;
}

public static (StatePart2 NextState, bool WasWall) ApplyStepPart2(StatePart2 state, Position delta) {
    var nextPosition = state.Position + Delta(state.Direction);
    var nextCube = state.Cube;
    var maxX = nextCube.Front.Keys.Max(p => p.X);
    var maxY = nextCube.Front.Keys.Max(p => p.Y);
    if (nextPosition.X < 0) {
        nextCube = nextCube.Rotate(CubeRotation.Right);
        nextPosition.X = maxX;
    }
    if (nextPosition.X > maxX) {
        nextCube = nextCube.Rotate(CubeRotation.Left);
        nextPosition.X = 0;
    }
    if (nextPosition.Y < 0) {
        nextCube = nextCube.Rotate(CubeRotation.Down);
        nextPosition.Y = maxY;
    }
    if (nextPosition.Y > maxY) {
        nextCube = nextCube.Rotate(CubeRotation.Up);
        nextPosition.Y = 0;
    }
    return nextCube.Front[nextPosition].Content switch {
        Space.Wall => (state, true),
        Space.Empty => (state with { Cube = nextCube, Position = nextPosition}, false),
    };
}

public static StatePart2 ApplyWalkInstructionPart2(StatePart2 state, int walkAmount) {
    var currentState = state;
    for (int i = 0; i < walkAmount; i++) {
        var (nextState, wasWall) = ApplyStepPart2(currentState, Delta(currentState.Direction));
        if (wasWall) {
            break;
        } else {
            currentState = nextState;
        }
    }
    return currentState;
}

public static StatePart2 ApplyTurnInstructionPart2(StatePart2 state, TurnDirection turn) {
   return state with {
        Direction = ApplyTurn(state.Direction, turn),
    };
}

public static StatePart2 ApplyInstructionPart2(StatePart2 state, Instruction instruction) {
    if (instruction.WalkAmount != null) {
        return ApplyWalkInstructionPart2(state, instruction.WalkAmount.Value);
    }
    if (instruction.TurnDirection != null) {
        return ApplyTurnInstructionPart2(state, instruction.TurnDirection.Value);
    }
    throw new InvalidOperationException("Invalid instruction");
}

public static Direction Reorient(Cube cube, Direction direction) {
    var faceOrientationVector = cube.Front[new Position { X = 1, Y = 0 }].MapPosition
        - cube.Front[new Position { X = 0, Y = 0 }].MapPosition;
    var numberOfExtraLeftTurnsToReOrient = (faceOrientationVector.X, faceOrientationVector.Y) switch {
        (1, 0) => 0,
        (0, 1) => 1,
        (-1, 0) => 2,
        (0, -1) => 3,
    };
    return Enumerable.Repeat(TurnDirection.Left, numberOfExtraLeftTurnsToReOrient)
        .Aggregate(direction, (newDirection, turn) => ApplyTurn(newDirection, turn));
}

In [281]:
// Part 2
var (map, instructions) = ParseInput(input);
var cube = CubeFromMap(map, sideLength);
var startingState = new StatePart2 {
    Position = new Position { X = 0, Y = 0 },
    Direction = Direction.Right,
    Cube = cube,
};

var finalState = instructions
    .Aggregate(startingState, (state, instruction) => ApplyInstructionPart2(state, instruction));
var finalPosition = finalState.Cube.Front[finalState.Position].MapPosition;
var finalDirection = Reorient(finalState.Cube, finalState.Direction);
(finalPosition.Y + 1) * 1000 + (finalPosition.X + 1) * 4 + Value(finalDirection)

34426